# Solution Architecture to Element Tables

The below code is designed to convert two exports of data (shapes, edges) from the solution architecture Visio diagrams and get it into an element table, which lists all the shapes, their type, and relationship.

Version 2: trying to address some of the outstanding problems in the original code.

Author: Yuval Berman, July 2023

Outstanding problems (in approximate order of importance & effort to fix)

### Done:
- Is the shape ID correct, considering there was a double up in the infrastructure table between a couple of shapes? Maybe look at matching based on all of "Shape Name" rather than just the ID - **done, that worked**
- "Type" may have to change where it is inferred from, as the Infrastructure table broke it. It may be should be inferred from Master, or Archimate Type? **done, it is now based on "ArchiMateType" column**
- Combined element tables for processes - **done, they need to be on the same page though**
- Understanding why some have double entries and fixing it **the solution for combining element tables should address this**
- Merge shapes that have the same ID / name? - **should be done based on solution for combining element tables**
- If there is no relationship, add "N/A" - **done**
- Remove newlines ('\n') in shape names - **done**
- If there are arrows that face both ways, is there a way to record this relationship? Is this data available in our exports? - Yes, in "Directionality" column - add this to the code. **done**


### To be done:
- _Existing / New / Modified - may not be able to do_
- _If there are arrows that face both ways, is there a way to record this relationship? Is this data available in our exports? - Yes, in "Directionality" column - add this to the code._
- _Remove arrow lables / floating shapes (e.g. HTTPS) from showing up in the final elements table - may not be able to do_
- _Filter out "Groups"_
- Sort alphabetically
- Maybe find a way to automate some of the common errors??
There also may be errors that are hard to pick up / address (such as the Master Name being used for EndX rather than the Shape Name...)


This should be ready to plug and play now. If the diagrams were perfect, then there is potential for automating it further so it does all files in a directory, but the fact that we have to filter the errors makes this harder.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.listdir("C:/Users/MB273QG/Downloads/DLPShapesExports/")

['BBABusinessFunction.csv',
 'BBAManageDataClassification.csv',
 'BBAManageDLPFrameworkProcesses.csv',
 'BBAManageSecurityIncidents.csv',
 'TBABusinessFunction.csv',
 'TBADocumentLabellingAndProtection.csv',
 'TBADocumentTransmissionSharing.csv',
 'TBAManageCloudUsage.csv',
 'TBAManageDataClassification.csv',
 'TBAManageDLPFrameworkProcesses.csv',
 'TBAManageDLPPlatform.csv',
 'TBAManageDLPPolicies.csv',
 'TBAManageDLPProfiles.csv',
 'TBAManageDLPReporting.csv',
 'TBAManageDLPSupportingInfrastructure.csv',
 'TBAManageSecurityIncidents.csv',
 'TBAManageSensitivityLabelPlatform.csv']

In [118]:
# Read in the data, don't forget to change path name when needed.

name = 'TBAManageCloudUsage'

shapes = pd.read_csv("C:/Users/MB273QG/Downloads/DLPShapesExports/"+name+".csv",skiprows = 1)
arrows = pd.read_excel("C:/Users/MB273QG/Downloads/DLPEdgesExports/"+name+".xlsx",'Here')
#arrows = pd.read_csv("C:/Users/MB273QG/Downloads/EdgesExports/TargetInfraNet1.csv")

In [119]:
def initial_data_cleaning(shapes,arrows):
    # Get the relevant information about the arrows
    arrows = arrows.dropna(how='any')
    beginarrow = [(x+"(.X!").split('(')[2].split('!')[0] for x in list(arrows['BeginX'])]
    endarrow = [(x+"(.X!").split('(')[2].split('!')[0] for x in list(arrows['EndX'])]
    
    arrows['BeginShape'] = beginarrow
    arrows['EndShape']   = endarrow 


    # Merge Arrows and Shapes DFs
    arrows = arrows.merge(shapes, left_on='ShapeKey', right_on='Shape ID', how='left')
    arrows['Relationship'] = [x.split('.')[0] for x in list(arrows['Shape Name'])]
    arrows.loc[arrows["Directionality"] == "Both Directions", "Relationship"] = "AccessBoth" 
    arrows = arrows.dropna(axis=1)
    arrows = arrows.drop(columns = ['Height', 'Shape ID', 'Width', 'X Location', 'Y Location', 'ArchiMateLayer', 'Master Name','ArchiMateType'])

    # Get the relevant information about shapes
    shapes = shapes.dropna(subset = ['Master Name', 'Displayed Text'])
    shapes = shapes.dropna(axis=1)
    shapes = shapes.reset_index(drop=True)
    
    # Get rid of newline characters
    text = list(shapes['Displayed Text'])
    newtext = []
    for item in list(shapes['Displayed Text']):
        newtext.append(item.replace("\n"," "))
    shapes['Displayed Text'] = newtext
    
    return shapes, arrows

In [120]:
shapes, arrows = initial_data_cleaning(shapes,arrows)

C:\Users\MB273QG\AppData\Local\Temp/ipykernel_27012/3507410024.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrows['BeginShape'] = beginarrow
C:\Users\MB273QG\AppData\Local\Temp/ipykernel_27012/3507410024.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arrows['EndShape']   = endarrow


Now that everything has been initialised, there are some problems we may have to solve if there are issues in the data.'

Firstly, check the arrows database to see if there is an "X" in either the BeginShape or EndShape columns. If this is the case it means that the arrow wasn't recorded properly (potentially not being linked to the shape correctly). You can go directly into the drawing to fix this, or you can edit the data directly. To do this: 
1. Work out the name of the shape that the arrow starts / ends in. Do this by using the .loc command on the ID of the shape.
2. Look at the original Visio diagram to see if you can work out the arrow that is being problematic, based on the shape and arrow relationship (if you can't tell which one, work out if the other shapes around it have their arrows recorded properly)
3. Use the .loc command again to work out the ID of the shape that wasn't recorded properly (however now find it based on the "Displayed Text" field.
4. Change the data to fit the format of others, with the new shape ID you found.
5. Read the data in again.

In [121]:
pd.set_option('max_row', None)

In [4]:
#arrows

In [123]:
shapes.loc[shapes["Shape Name"] == 'Business Process.1135']

,Displayed Text,Height,Master Name,Shape ID,Shape Name,Width,X Location,Y Location,ArchiMateAspect,ArchiMateLayer,...,IconWidth,ShapeLayout,ShowLayer,TextHeight,TextLayout,TextLocPinX,TextLocPinY,TextPinX,TextPinY,TextWidth
24,Tag as sanctioned in the DLP solution,19.05 mm,Business Process.5006,1135,Business Process.1135,31.75 mm,244.16 mm,318.08 mm,Behavior,Business Layer,...,4.57 mm,1.0,0.0,19.05 mm,3.0,15.88 mm,9.53 mm,15.88 mm,9.53 mm,31.75 mm


In [124]:
shapes.loc[shapes["Displayed Text"] == 'App access is restricted']

,Displayed Text,Height,Master Name,Shape ID,Shape Name,Width,X Location,Y Location,ArchiMateAspect,ArchiMateLayer,...,IconWidth,ShapeLayout,ShowLayer,TextHeight,TextLayout,TextLocPinX,TextLocPinY,TextPinX,TextPinY,TextWidth
16,App access is restricted,19.05 mm,Business event,1091,Business event.1091,31.75 mm,293.72 mm,478.08 mm,Behavior,Business Layer,...,4.57 mm,1.0,0.0,19.05 mm,3.0,15.88 mm,9.53 mm,15.88 mm,9.53 mm,31.75 mm
28,App access is restricted,19.05 mm,Business event,1159,Business event.1159,31.75 mm,292.41 mm,318.08 mm,Behavior,Business Layer,...,4.57 mm,1.0,0.0,19.05 mm,3.0,15.88 mm,9.53 mm,15.88 mm,9.53 mm,31.75 mm


In [125]:
arrows.loc[arrows["BeginShape"] == 'Business Process.12205']

,ShapeKey,BeginX,EndX,BeginShape,EndShape,Shape Name,Relationship


In [126]:
# Define arrow relationships

relation_start = {"Composition": "Composes", 
                  "Aggregation" : "Aggregates", 
                  "Assignment": "Assigns", 
                  "Realization": "Realises", 
                  "Serving":"Serves", 
                  "Access":"Accesses", 
                  "Influence": "Influences", 
                  "Association" : "Associated with", 
                  "Triggering": "Triggers",
                  "Flow": "Flows to", 
                  "Specialization" : "Specialises",
                 "AccessBoth" : "Access (both ways)"}

relation_end = {"Composition": "Composed of", 
                "Aggregation" : "Aggregated from", 
                "Assignment": "Assigned by", 
                  "Realization": "Realised by", 
                "Serving":"Served by", 
                "Access":"Accessed by", 
                "Influence": "Influenced by", 
                  "Association" : "Associated with", 
                "Triggering": "Triggered by", 
                "Flow": "Flows from", 
                  "Specialization" : "Specialises",
               "AccessBoth": "Access (both ways)"}

Now, let's start initialising the elements table!

In [127]:
def create_elements_table(shapes,arrows):
    
    oneline = []
    for sh in list(shapes["Shape Name"]):
        relations = []
        related_shape =[]
        all_arrows_start = arrows.loc[arrows["BeginShape"] == sh]
        all_arrows_end = arrows.loc[arrows["EndShape"] == sh]
        status = "Existing"
        shapetext = list(shapes.loc[shapes["Shape Name"] == sh]["Displayed Text"])[0]
        shapetype = list(shapes.loc[shapes["Shape Name"] == sh]["ArchiMateType"])[0]
        if len(all_arrows_end)==0 and len(all_arrows_start)==0:
            oneline.append([status,shapetext,shapetype,None,"N/A","N/A"])
        for i in range(len(all_arrows_end)):
            item = list(all_arrows_end["Relationship"])[i]
            shapeid = list(all_arrows_end["BeginShape"])[i]
            relationship = relation_end[item]
            related_shape = list(shapes.loc[shapes["Shape Name"] == shapeid]["Displayed Text"])[0]
            oneline.append([status,shapetext,shapetype,None,relationship,related_shape])
        for i in range(len(all_arrows_start)):
            item = list(all_arrows_start["Relationship"])[i]
            shapeid = list(all_arrows_start["EndShape"])[i]
            relationship = relation_start[item]
            related_shape = list(shapes.loc[shapes["Shape Name"] == shapeid]["Displayed Text"])[0]
            oneline.append([status,shapetext,shapetype,None,relationship,related_shape])
    
    elem = pd.DataFrame(oneline, columns = ['Status','Name', 'Type', 'Description', 'Relationship','Related to'] )
    elem["ID"] = elem["Name"]+ elem["Type"]
    
    return elem

In [128]:
elem = create_elements_table(shapes, arrows)

In [129]:
newline = []
for item in set(list(elem["ID"])):
    temp = elem.loc[elem["ID"] == item]
    
    newstatus = list(temp["Status"])[0]
    newname = list(temp["Name"])[0]
    newtype = list(temp["Type"])[0]
    
    temp["RelKey"] = temp["Relationship"] + temp["Related to"]
    temp = temp.drop_duplicates(subset=["RelKey"])
    
    n = 1
    newrel = ''
    newshape = ''
    
    if len(temp) >1:
        for i in range(len(temp)):
            if list(temp["Relationship"])[i] != "N/A":
                newrel += str(n) +'. ' + list(temp["Relationship"])[i]
                newshape += str(n) + '. ' + list(temp["Related to"])[i]
                if i < len(temp) - 1:
                    newrel += '\n'
                    newshape += '\n'
                n+=1
    else:
        newrel = list(temp["Relationship"])[0]
        newshape = list(temp["Related to"])[0]
    
    newline.append([newstatus,newname,newtype,None,newrel,newshape])
    
newelem = pd.DataFrame(newline, columns = ['Status','Name', 'Type', 'Description', 'Relationship','Related to'] )

C:\Users\MB273QG\AppData\Local\Temp/ipykernel_27012/2691430400.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["RelKey"] = temp["Relationship"] + temp["Related to"]


In [3]:
#newelem

Done! Now, save the file (make sure to change path name each time).

In [131]:
newelem.to_csv("C:/Users/MB273QG/Downloads/DLPResultsExports/"+name+".csv")